In [ ]:
import pypsa
import pandas as pd
from pypsa.descriptors import get_switchable_as_dense as as_dense
import plotly.graph_objects as go
import numpy as np
import yaml
import os
from matplotlib.colors import to_rgba

In [ ]:
PATH = "../../../playgrounds/pr/pypsa-eur-sec/"
SCENARIO = "elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030"
RUN = "20211218-181-h2"

OUTPUT = "../results/graphics/"
OUTPUT_SCENARIO = f"{OUTPUT}/{RUN}/{SCENARIO}/"

In [ ]:
if not os.path.exists(OUTPUT_SCENARIO):
    os.makedirs(OUTPUT_SCENARIO)

In [ ]:
with open(PATH + "config.yaml") as file:
    config = yaml.safe_load(file)

colors = config["plotting"]["tech_colors"]

colors["electricity grid"] = colors["electricity"]
colors["ground-sourced ambient"] = colors["ground heat pump"]
colors["air-sourced ambient"] = colors["air heat pump"]
colors["co2 atmosphere"] = colors["co2"]
colors["co2 stored"] = colors["co2"]
colors["net co2 emissions"] = colors["co2"]
colors["co2 sequestration"] = colors["co2"]
colors["fossil oil"] = colors["oil"]
colors["fossil gas"] = colors["gas"]
colors["biogas to gas"] = colors["biogas"]
colors["process emissions from feedstocks"] = colors["process emissions"]

gas_boilers = [
    "residential rural gas boiler",
    "services rural gas boiler",
    "residential urban decentral gas boiler",
    "services urban decentral gas boiler",
    "urban central gas boiler",
]
for gas_boiler in gas_boilers:
    colors[gas_boiler] = colors["gas boiler"]

colors["urban central gas CHP"] = colors["CHP"]
colors["urban central gas CHP CC"] = colors["CHP"]
colors["urban central solid biomass CHP"] = colors["CHP"]
colors["urban central solid biomass CHP CC"] = colors["CHP"]

In [ ]:
n = pypsa.Network(f"{PATH}results/{RUN}/postnetworks/{SCENARIO}.nc")

In [ ]:
fn = f"{PATH}resources/industrial_energy_demand_elec_s_181_2030.csv"
feedstock_emissions = (
    pd.read_csv(fn, index_col=0)["process emission from feedstock"].sum() * 1e6
)  # t

In [ ]:
def prepare_sankey(n):
    columns = ["label", "source", "target", "value"]

    gen = (
        (n.snapshot_weightings.generators @ n.generators_t.p)
        .groupby(
            [
                n.generators.carrier,
                n.generators.carrier,
                n.generators.bus.map(n.buses.carrier),
            ]
        )
        .sum()
        .div(1e6)
    )  # TWh

    gen.index.set_names(columns[:-1], inplace=True)
    gen = gen.reset_index(name="value")
    gen = gen.loc[gen.value > 0.1]

    gen["source"] = gen["source"].replace({"gas": "fossil gas", "oil": "fossil oil"})

    sto = (
        (n.snapshot_weightings.generators @ n.stores_t.p)
        .groupby(
            [n.stores.carrier, n.stores.carrier, n.stores.bus.map(n.buses.carrier)]
        )
        .sum()
        .div(1e6)
    )
    sto.index.set_names(columns[:-1], inplace=True)
    sto = sto.reset_index(name="value")
    sto = sto.loc[sto.value > 0.1]

    su = (
        (n.snapshot_weightings.generators @ n.storage_units_t.p)
        .groupby(
            [
                n.storage_units.carrier,
                n.storage_units.carrier,
                n.storage_units.bus.map(n.buses.carrier),
            ]
        )
        .sum()
        .div(1e6)
    )
    su.index.set_names(columns[:-1], inplace=True)
    su = su.reset_index(name="value")
    su = su.loc[su.value > 0.1]

    load = (
        (n.snapshot_weightings.generators @ as_dense(n, "Load", "p_set"))
        .groupby([n.loads.carrier, n.loads.carrier, n.loads.bus.map(n.buses.carrier)])
        .sum()
        .div(1e6)
        .swaplevel()
    )  # TWh
    load.index.set_names(columns[:-1], inplace=True)
    load = load.reset_index(name="value")

    load = load.loc[~load.label.str.contains("emissions")]
    load.target += " demand"

    for i in range(5):
        n.links[f"total_e{i}"] = (
            n.snapshot_weightings.generators @ n.links_t[f"p{i}"]
        ).div(
            1e6
        )  # TWh
        n.links[f"carrier_bus{i}"] = n.links[f"bus{i}"].map(n.buses.carrier)

    def calculate_losses(x):
        energy_ports = x.loc[
            x.index.str.contains("carrier_bus") & ~x.str.contains("co2", na=False)
        ].index.str.replace("carrier_bus", "total_e")
        return -x.loc[energy_ports].sum()

    n.links["total_e5"] = n.links.apply(calculate_losses, axis=1)
    n.links["carrier_bus5"] = "losses"

    df = pd.concat(
        [
            n.links.groupby(["carrier", "carrier_bus0", "carrier_bus" + str(i)]).sum()[
                "total_e" + str(i)
            ]
            for i in range(1, 6)
        ]
    ).reset_index()
    df.columns = columns

    # fix heat pump energy balance

    hp = n.links.loc[n.links.carrier.str.contains("heat pump")]

    hp_t_elec = n.links_t.p0.filter(like="heat pump")

    grouper = [hp["carrier"], hp["carrier_bus0"], hp["carrier_bus1"]]
    hp_elec = (
        (-n.snapshot_weightings.generators @ hp_t_elec)
        .groupby(grouper)
        .sum()
        .div(1e6)
        .reset_index()
    )
    hp_elec.columns = columns

    df = df.loc[~(df.label.str.contains("heat pump") & (df.target == "losses"))]

    df.loc[df.label.str.contains("heat pump"), "value"] -= hp_elec["value"].values

    df.loc[df.label.str.contains("air heat pump"), "source"] = "air-sourced ambient"
    df.loc[
        df.label.str.contains("ground heat pump"), "source"
    ] = "ground-sourced ambient"

    df = pd.concat([df, hp_elec])
    df = df.set_index(["label", "source", "target"]).squeeze()
    df = pd.concat(
        [
            df.loc[df < 0].mul(-1),
            df.loc[df > 0].swaplevel(1, 2),
        ]
    ).reset_index()
    df.columns = columns

    # make DAC demand
    df.loc[df.label == "DAC", "target"] = "DAC"

    to_concat = [df, gen, su, sto, load]
    connections = pd.concat(to_concat).sort_index().reset_index(drop=True)

    # aggregation

    src_contains = connections.source.str.contains
    trg_contains = connections.target.str.contains

    connections.loc[src_contains("low voltage"), "source"] = "AC"
    connections.loc[trg_contains("low voltage"), "target"] = "AC"
    connections.loc[src_contains("water tank"), "source"] = "water tank"
    connections.loc[trg_contains("water tank"), "target"] = "water tank"
    connections.loc[src_contains("solar thermal"), "source"] = "solar thermal"
    connections.loc[src_contains("battery"), "source"] = "battery"
    connections.loc[trg_contains("battery"), "target"] = "battery"
    connections.loc[src_contains("Li ion"), "source"] = "battery"
    connections.loc[trg_contains("Li ion"), "target"] = "battery"

    connections.loc[src_contains("heat") & ~src_contains("demand"), "source"] = "heat"
    connections.loc[trg_contains("heat") & ~trg_contains("demand"), "target"] = "heat"

    connections = connections.loc[
        ~(connections.source == connections.target)
        & ~connections.source.str.contains("co2")
        & ~connections.target.str.contains("co2")
        & ~connections.source.str.contains("emissions")
        & ~connections.source.isin(["gas for industry", "solid biomass for industry"])
        & (connections.value >= 0.5)
    ]

    where = connections.label == "urban central gas boiler"
    connections.loc[where] = connections.loc[where].replace("losses", "fossil gas")

    connections.replace("AC", "electricity grid", inplace=True)

    return connections

In [ ]:
def plot_sankey(connections, fn=None):
    labels = np.unique(connections[["source", "target"]])

    nodes = pd.Series({v: i for i, v in enumerate(labels)})

    node_colors = pd.Series(nodes.index.map(colors).fillna("grey"), index=nodes.index)

    link_colors = [
        "rgba{}".format(to_rgba(node_colors[src], alpha=0.5))
        for src in connections.source
    ]

    fig = go.Figure(
        go.Sankey(
            arrangement="snap",  # [snap, nodepad, perpendicular, fixed]
            valuesuffix="TWh",
            valueformat=".1f",
            node=dict(pad=20, thickness=20, label=nodes.index, color=node_colors),
            link=dict(
                source=connections.source.map(nodes),
                target=connections.target.map(nodes),
                value=connections.value,
                label=connections.label,
                color=link_colors,
            ),
        )
    )

    fig.update_layout(title=f"Sankey Diagram: {SCENARIO}", font_size=15)

    if fn is not None:
        fig.write_html(fn + ".html")
        fig.write_image(fn + ".pdf", width=1000, height=1000)

In [ ]:
connections = prepare_sankey(n)

In [ ]:
connections.to_csv(f"{OUTPUT_SCENARIO}sankey.csv")

In [ ]:
plot_sankey(connections, fn=f"{OUTPUT_SCENARIO}sankey")

## Carbon Sankey

In [ ]:
columns = ["label", "source", "target", "value"]

In [ ]:
collection = []

In [ ]:
# DAC
value = -(n.snapshot_weightings.generators @ n.links_t.p1.filter(like="DAC")).sum()
collection.append(
    pd.Series(
        dict(label="DAC", source="co2 atmosphere", target="co2 stored", value=value)
    )
)

In [ ]:
# process emissions
value = -(
    n.snapshot_weightings.generators @ n.links_t.p1.filter(regex="process emissions$")
).sum()
collection.append(
    pd.Series(
        dict(
            label="process emissions",
            source="process emissions",
            target="co2 atmosphere",
            value=value,
        )
    )
)

In [ ]:
# process emissions CC
value = -(
    n.snapshot_weightings.generators @ n.links_t.p1.filter(regex="process emissions CC")
).sum()
collection.append(
    pd.Series(
        dict(
            label="process emissions CC",
            source="process emissions",
            target="co2 atmosphere",
            value=value,
        )
    )
)

value = -(
    n.snapshot_weightings.generators @ n.links_t.p2.filter(regex="process emissions CC")
).sum()
collection.append(
    pd.Series(
        dict(
            label="process emissions CC",
            source="process emissions",
            target="co2 stored",
            value=value,
        )
    )
)

In [ ]:
# OCGT
value = -(n.snapshot_weightings.generators @ n.links_t.p2.filter(like="OCGT")).sum()
collection.append(
    pd.Series(dict(label="OCGT", source="gas", target="co2 atmosphere", value=value))
)

In [ ]:
# Sabatier
value = (n.snapshot_weightings.generators @ n.links_t.p2.filter(like="Sabatier")).sum()
collection.append(
    pd.Series(dict(label="Sabatier", source="co2 stored", target="gas", value=value))
)

In [ ]:
# SMR
value = -(n.snapshot_weightings.generators @ n.links_t.p2.filter(regex="SMR$")).sum()
collection.append(
    pd.Series(dict(label="SMR", source="gas", target="co2 atmosphere", value=value))
)

In [ ]:
# SMR CC
value = -(n.snapshot_weightings.generators @ n.links_t.p2.filter(regex="SMR CC")).sum()
collection.append(
    pd.Series(dict(label="SMR CC", source="gas", target="co2 atmosphere", value=value))
)

value = -(n.snapshot_weightings.generators @ n.links_t.p3.filter(like="SMR CC")).sum()
collection.append(
    pd.Series(dict(label="SMR CC", source="gas", target="co2 stored", value=value))
)

In [ ]:
# gas boiler
gas_boilers = [
    "residential rural gas boiler",
    "services rural gas boiler",
    "residential urban decentral gas boiler",
    "services urban decentral gas boiler",
    "urban central gas boiler",
]
for gas_boiler in gas_boilers:
    value = -(
        n.snapshot_weightings.generators @ n.links_t.p2.filter(like=gas_boiler)
    ).sum()
    collection.append(
        pd.Series(
            dict(label=gas_boiler, source="gas", target="co2 atmosphere", value=value)
        )
    )

In [ ]:
# biogas to gas
value = (
    n.snapshot_weightings.generators @ n.links_t.p2.filter(like="biogas to gas")
).sum()
collection.append(
    pd.Series(
        dict(
            label="biogas to gas", source="co2 atmosphere", target="biogas", value=value
        )
    )
)
collection.append(
    pd.Series(dict(label="biogas to gas", source="biogas", target="gas", value=value))
)

In [ ]:
# solid biomass for industry CC
value = (
    n.snapshot_weightings.generators
    @ n.links_t.p2.filter(like="solid biomass for industry CC")
).sum()
collection.append(
    pd.Series(
        dict(
            label="solid biomass for industry CC",
            source="co2 atmosphere",
            target="solid biomass",
            value=value,
        )
    )
)

value = -(
    n.snapshot_weightings.generators
    @ n.links_t.p3.filter(like="solid biomass for industry CC")
).sum()
collection.append(
    pd.Series(
        dict(
            label="solid biomass for industry CC",
            source="solid biomass",
            target="co2 stored",
            value=value,
        )
    )
)

In [ ]:
# gas for industry
value = -(
    n.snapshot_weightings.generators @ n.links_t.p1.filter(regex="gas for industry$")
).sum()
collection.append(
    pd.Series(
        dict(
            label="gas for industry", source="gas", target="co2 atmosphere", value=value
        )
    )
)

In [ ]:
# gas for industry CC
value = -(
    n.snapshot_weightings.generators @ n.links_t.p2.filter(like="gas for industry CC")
).sum()
collection.append(
    pd.Series(
        dict(
            label="gas for industry CC",
            source="gas",
            target="co2 atmosphere",
            value=value,
        )
    )
)

value = -(
    n.snapshot_weightings.generators @ n.links_t.p3.filter(like="gas for industry CC")
).sum()
collection.append(
    pd.Series(
        dict(
            label="gas for industry CC", source="gas", target="co2 stored", value=value
        )
    )
)

In [ ]:
# Fischer-Tropsch
value = (
    n.snapshot_weightings.generators @ n.links_t.p2.filter(like="Fischer-Tropsch")
).sum()
collection.append(
    pd.Series(
        dict(label="Fischer-Tropsch", source="co2 stored", target="oil", value=value)
    )
)

In [ ]:
# urban central gas CHP
value = -(
    n.snapshot_weightings.generators @ n.links_t.p3.filter(like="urban central gas CHP")
).sum()
collection.append(
    pd.Series(
        dict(
            label="urban central gas CHP",
            source="gas",
            target="co2 atmosphere",
            value=value,
        )
    )
)

In [ ]:
# urban central gas CHP CC
tech = "urban central gas CHP CC"
value = -(n.snapshot_weightings.generators @ n.links_t.p3.filter(like=tech)).sum()
collection.append(
    pd.Series(dict(label=tech, source="gas", target="co2 atmosphere", value=value))
)

value = -(n.snapshot_weightings.generators @ n.links_t.p4.filter(like=tech)).sum()
collection.append(
    pd.Series(dict(label=tech, source="gas", target="co2 stored", value=value))
)

In [ ]:
# urban solid biomass CHP CC
tech = "urban central solid biomass CHP CC"

value = (n.snapshot_weightings.generators @ n.links_t.p3.filter(like=tech)).sum()
collection.append(
    pd.Series(
        dict(label=tech, source="co2 atmosphere", target="solid biomass", value=value)
    )
)

value = -(n.snapshot_weightings.generators @ n.links_t.p4.filter(like=tech)).sum()
collection.append(
    pd.Series(
        dict(label=tech, source="solid biomass", target="co2 stored", value=value)
    )
)

In [ ]:
# oil emissions
value = -(
    n.snapshot_weightings.generators
    @ as_dense(n, "Load", "p_set").filter(regex="^oil emissions", axis=1)
).sum()
collection.append(
    pd.Series(
        dict(label="oil emissions", source="oil", target="co2 atmosphere", value=value)
    )
)

In [ ]:
# agriculture machinery oil emissions
value = -(
    n.snapshot_weightings.generators
    @ as_dense(n, "Load", "p_set").filter(
        like="agriculture machinery oil emissions", axis=1
    )
).sum()
collection.append(
    pd.Series(
        dict(
            label="agriculture machinery oil emissions",
            source="oil",
            target="co2 atmosphere",
            value=value,
        )
    )
)

In [ ]:
# feedstock_emissions
collection.append(
    pd.Series(
        dict(
            label="process emissions from feedstocks",
            source="oil",
            target="process emissions",
            value=feedstock_emissions,
        )
    )
)

In [ ]:
df = pd.concat(collection, axis=1).T
df.value /= 1e6  # Mt

In [ ]:
# fossil gas
co2_intensity = 0.2  # t/MWh
value = (n.snapshot_weightings.generators @ n.generators_t.p.filter(like="gas")).div(
    1e6
).sum() * co2_intensity
row = pd.DataFrame(
    [dict(label="fossil gas", source="fossil gas", target="gas", value=value)]
)
df = pd.concat([df, row], axis=0)

In [ ]:
# fossil oil
co2_intensity = 0.27  # t/MWh
value = (n.snapshot_weightings.generators @ n.generators_t.p.filter(like="oil")).div(
    1e6
).sum() * 0.27
row = pd.DataFrame(
    [dict(label="fossil oil", source="fossil oil", target="oil", value=value)]
)
df = pd.concat([df, row], axis=0)

In [ ]:
# sequestration
value = (
    df.loc[df.target == "co2 stored", "value"].sum()
    - df.loc[df.source == "co2 stored", "value"].sum()
)
row = pd.DataFrame(
    [
        dict(
            label="co2 sequestration",
            source="co2 stored",
            target="co2 sequestration",
            value=value,
        )
    ]
)
df = pd.concat([df, row], axis=0)

In [ ]:
# net co2 emissions
value = (
    df.loc[df.target == "co2 atmosphere", "value"].sum()
    - df.loc[df.source == "co2 atmosphere", "value"].sum()
)
row = pd.DataFrame(
    [
        dict(
            label="net co2 emissions",
            source="co2 atmosphere",
            target="net co2 emissions",
            value=value,
        )
    ]
)
df = pd.concat([df, row], axis=0)

In [ ]:
def plot_carbon_sankey(co2, fn=None):
    labels = np.unique(co2[["source", "target"]])

    nodes = pd.Series({v: i for i, v in enumerate(labels)})

    node_colors = pd.Series(nodes.index.map(colors).fillna("grey"), index=nodes.index)

    link_colors = [
        "rgba{}".format(to_rgba(colors[src], alpha=0.5)) for src in co2.label
    ]

    fig = go.Figure(
        go.Sankey(
            arrangement="freeform",  # [snap, nodepad, perpendicular, fixed]
            valuesuffix=" MtCO2",
            valueformat=".1f",
            node=dict(pad=5, thickness=5, label=nodes.index, color=node_colors),
            link=dict(
                source=co2.source.map(nodes),
                target=co2.target.map(nodes),
                value=co2.value,
                label=co2.label,
                color=link_colors,
            ),
        )
    )

    fig.update_layout(
        title=f"Carbon Flow Sankey:<br>{SCENARIO}",
    )

    if fn is not None:
        fig.write_html(fn + ".html")
        fig.write_image(fn + ".pdf", width=1200, height=1000)

In [ ]:
df.to_csv(f"{OUTPUT_SCENARIO}sankey-carbon.csv")

In [ ]:
plot_carbon_sankey(df, fn=f"{OUTPUT_SCENARIO}sankey-carbon")